In [8]:
!pip install dash  
# !pip install dash==1.19.0
!pip install jupyter_dash
!pip install --upgrade plotly
!pip install dash --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
# Import required libraries
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update


# Create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True


In [10]:

spacex_df =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

max_value = spacex_df['Payload Mass (kg)'].max()
min_value = spacex_df['Payload Mass (kg)'].min()

In [11]:

# Create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center', 'color': '#503D36',   'font-size': 40}), 
                                # Create an outer division 
            html.Div([
                html.Div([   
                    # TODO2: Add a dropdown
                dcc.Dropdown(
                    id='site-dropdown', 
                    options=[
                            {'label': 'All Sites', 'value': 'ALL'},
                            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                            ],
                    value='ALL',  
                    placeholder='Select a Launch Site',
                    style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'},
                    searchable=True 
                ) ,  
          html.Div(id='output-graph1'),
          html.Div(
              dcc.RangeSlider(
                  id='Payload range (kg)',
                  min=0, max=10000, step=1000,
                  marks={0: '0',
                        100: '100'},
                  value=[min_value, max_value]
                  ),   
          ),
            ]),
          html.Div(id='output-graph2')
        ])
    ])
# Function decorator to specify function input and output
@app.callback( 
    [Output('output-graph1','children'),
     Output('output-graph2','children')],   
    [Input( 'site-dropdown', 'value') ,
     Input( 'Payload range (kg)', 'value'),]  
) 

def update_graph(entered_site,payload):
    filtered_df = spacex_df 
 
    filtered_dfpay = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]


    if entered_site == 'ALL':
        
        filtered_df =   filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count') 
        fig =  dcc.Graph(
            figure=   px.pie(filtered_df,values='class count',names='Launch Site',title=f"Total Success Launches")
        )
        fig0 =   dcc.Graph( 
            figure =px.scatter(filtered_dfpay,
                               x='Payload Mass (kg)',
                               y='class',
                               color='Booster Version Category',
                               title='Success count on Payload mass for all sites') 
        ) 
        return  [fig, fig0] 
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site']== entered_site] 

        filtered_df = filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = dcc.Graph( 
            figure =    px.pie(filtered_df,values='class count',
                               names='class',title=f"Total Success Launches for site {entered_site}") 
            )
        fig0 =   dcc.Graph( 
        figure = px.scatter(filtered_dfpay[filtered_dfpay['Launch Site']==entered_site],
                            x='Payload Mass (kg)',y='class',
                            color='Booster Version Category',
                            title=f"Success count on Payload mass for the site {entered_site}")
        ) 
        return [fig, fig0]   

# Run the app
if __name__ == '__main__':
    # REVIEW8: Adding dev_tools_ui=False, dev_tools_props_check=False can prevent error appearing before calling callback function
    app.run_server(mode="inline", host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False) 

Dash is running on http://localhost:8050/



INFO:dash.dash:Dash is running on http://localhost:8050/

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:8050
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/May/2023 07:09:54] "GET /_alive_bcd19793-e67d-4c8b-9be8-6c0506b0d32b HTTP/1.1" 200 -


<IPython.core.display.Javascript object>